Ethics of AI Final Project

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.2 MB/s eta 0:00:00


In [17]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import  DecisionTreeClassifier ,plot_tree, export_text# Decision tree algorithm and plotting functions for the Decision tree
from matplotlib import pyplot as plt # plotting/graphing
import numpy as np
from itertools import product


In [49]:
# fetch dataset and apply processing file
dataset = load_dataset("mstz/speeddating",trust_remote_code=True)["train"]

# convert to df
df = pd.DataFrame(dataset)


# split into x and y sets
X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date']) # is match is our target variable, which is determined from the last 2 so we need to drop all
X = pd.get_dummies(X) #get dummies for X
y = df.is_match

X.columns


Index(['is_dater_male', 'dater_age', 'dated_age', 'age_difference',
       'are_same_race', 'same_race_importance_for_dater',
       'same_religion_importance_for_dater',
       'attractiveness_importance_for_dated', 'sincerity_importance_for_dated',
       'intelligence_importance_for_dated', 'humor_importance_for_dated',
       'ambition_importance_for_dated',
       'shared_interests_importance_for_dated',
       'attractiveness_score_of_dater_from_dated',
       'sincerity_score_of_dater_from_dated',
       'intelligence_score_of_dater_from_dated',
       'humor_score_of_dater_from_dated', 'ambition_score_of_dater_from_dated',
       'shared_interests_score_of_dater_from_dated',
       'attractiveness_importance_for_dater', 'sincerity_importance_for_dater',
       'intelligence_importance_for_dater', 'humor_importance_for_dater',
       'ambition_importance_for_dater',
       'shared_interests_importance_for_dater',
       'self_reported_attractiveness_of_dater',
       'self_repor

In [3]:
len(df)

1048

In [50]:
# split into training and tests, ALL RANDOM STATES SET TO 0
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .33,random_state=0)

In [51]:
# Different models
# Random Forest

#Set estimators as 100, random state 0, samples of at least 2 in each leaf
clf = RandomForestClassifier(n_estimators = 125, random_state=0, min_samples_leaf=2)
clf.fit(x_train, y_train) # fit model

# get the roc auc and pring
cross_val_accuracy_roc_auc = (cross_val_score(clf, x_train, y_train, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_accuracy_roc_auc

83.31332309553726

# 1 Pre Processing
Our dataset has a variety of features. We will want to simplify our dataset to have a few key features:
Age, age difference for the pairing (Sensitive)
Race, same race for the pairing (Sensitive)
expected_number_of_likes_of_dater_from_20_people (non sensitive)
Already met before (non sensitive)


In [43]:
features = ['dater_age','dated_age','age_difference','dater_race','dated_race','are_same_race','expected_number_of_likes_of_dater_from_20_people','already_met_before']

X = df[features].copy()
y = df.is_match.copy()

# We will want to see where the race is the same or not

print('Matches where pairings are the same race: ',len(X[X['are_same_race']]))
print('Matches where pairings are not the same race: ',len(X[-X['are_same_race']]))

Matches where pairings are the same race:  437
Matches where pairings are not the same race:  611


In [44]:
print('Matches where pairings are the same race and match: ',len(X[X['are_same_race'] & y==1]))
print('Matches where pairings are not the same race and match: ',len(X[-X['are_same_race'] & y==1]))

Matches where pairings are the same race and match:  84
Matches where pairings are not the same race and match:  102


# Split data and Examine Match Rates by Group

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state= 372)

print('% Same Race in train: ', len(X_train[X_train['are_same_race']])/len(X_train))
print('% Not Same Race in train: ', len(X_train[-X_train['are_same_race']])/len(X_train))

print('%  Same Race in test: ', len(X_test[X_test['are_same_race']])/len(X_test))
print('% Not Same Race test: ', len(X_test[-X_test['are_same_race']])/len(X_test))

#make a df where the couple is a match
train_is_match = X_train.join(y_train)
train_is_match = train_is_match[y_train==1]

print('Difference in Match Rate between Groups in Training Data: ',
    ( (len(train_is_match[train_is_match['are_same_race']]) / len(X_train[X_train['are_same_race']]))
     -
    ( (len(train_is_match[-train_is_match['are_same_race']])) / len(X_train[-X_train['are_same_race']])))

)

% Same Race in train:  0.4092769440654843
% Not Same Race in train:  0.5907230559345157
%  Same Race in test:  0.43492063492063493
% Not Same Race test:  0.5650793650793651
Difference in Match Rate between Groups in Training Data:  0.029622786759045422


Metrics by Race

In [46]:
results = x_test
y_hat = clf.predict(x_test)
results['predicted'] = y_hat
results['actual'] = y_test

In [10]:
results.head()

,is_dater_male,dater_age,dated_age,age_difference,are_same_race,same_race_importance_for_dater,same_religion_importance_for_dater,attractiveness_importance_for_dated,sincerity_importance_for_dated,intelligence_importance_for_dated,...,dater_race_'Latino/Hispanic American',dater_race_caucasian,dater_race_other,dated_race_'Asian/Pacific Islander/Asian-American',dated_race_'Black/African American',dated_race_'Latino/Hispanic American',dated_race_caucasian,dated_race_other,predicted,actual
413,True,25,26,1,False,1.0,3.0,20.0,25.0,20.0,...,0,0,0,0,0,0,1,0,0,NaN
311,True,25,27,2,False,9.0,6.0,10.0,10.0,30.0,...,0,0,0,0,0,0,1,0,0,NaN
142,False,21,30,9,False,9.0,6.0,20.0,20.0,25.0,...,0,0,0,0,0,0,1,0,0,NaN
330,True,24,26,2,True,1.0,1.0,20.0,25.0,20.0,...,0,1,0,0,0,0,1,0,0,NaN
875,True,27,35,8,False,6.0,8.0,10.0,32.0,20.0,...,0,1,0,0,0,1,0,0,0,NaN


In [28]:
dater_groups = ["dater_race_'Asian/Pacific Islander/Asian-American'","dater_race_'Black/African American'","dater_race_'Latino/Hispanic American'", 'dater_race_caucasian','dater_race_other']
dated_groups = ["dated_race_'Asian/Pacific Islander/Asian-American'","dated_race_'Black/African American'","dated_race_'Latino/Hispanic American'", 'dated_race_caucasian','dated_race_other']

def compute_metrics(info, group):
  metric_df = info.loc[info[group] == 1]
  pred_pos = sum(metric_df['predicted'])
  act_pos = sum(metric_df['actual'])

  positive_rate = pred_pos/len(metric_df)
  accuracy = len(metric_df[metric_df['predicted'] == metric_df['actual']])/len(metric_df)
  tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
  fp = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 1)])
  fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
  tn = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 0)])
  tpr = tp/(tp + fn + 1)
  fpr = fp/(fp + tn)
  precision = tp/(tp + fp+ 1)
  recall = tp/(tp + fn + 1)

  row = [group, accuracy, precision, recall, positive_rate, tpr, fpr]

  return row

column_names = ['group', 'accuracy', 'precision', 'recall', 'positive_rate', 'tpr', 'fpr']
metrics_df = pd.DataFrame(columns=column_names)
for group in dater_groups:
  row = compute_metrics(results, group)
  metrics_df.loc[len(metrics_df)] = row

for group in dated_groups:
  row = compute_metrics(results, group)
  metrics_df.loc[len(metrics_df)] = row

metrics_df

,group,accuracy,precision,recall,positive_rate,tpr,fpr
0,dater_race_'Asian/Pacific Islander/Asian-Ameri...,0.289474,0.000000,0.000000,0.026316,0.000000,0.000000
1,dater_race_'Black/African American',0.250000,0.500000,0.500000,0.250000,0.500000,0.000000
2,dater_race_'Latino/Hispanic American',0.222222,0.000000,0.000000,0.037037,0.000000,0.000000
3,dater_race_caucasian,0.265700,0.400000,0.181818,0.077295,0.181818,0.036364
4,dater_race_other,0.142857,0.000000,0.000000,0.071429,0.000000,0.000000
5,dated_race_'Asian/Pacific Islander/Asian-Ameri...,0.240000,0.000000,0.000000,0.066667,0.000000,0.052632
6,dated_race_'Black/African American',0.250000,0.666667,0.400000,0.083333,0.400000,0.000000
7,dated_race_'Latino/Hispanic American',0.185185,0.000000,0.000000,0.074074,0.000000,0.000000
8,dated_race_caucasian,0.270408,0.333333,0.100000,0.061224,0.100000,0.018868
9,dated_race_other,0.291667,0.000000,0.000000,0.083333,0.000000,0.000000


## 2 - Inprocess Bias Mitigation

In [29]:
df = pd.DataFrame(dataset)

X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date']) # is match is our target variable, which is determined from the last 2 so we need to drop all
X = pd.get_dummies(X) #get dummies for X
y = df.is_match

# Same race data split into y and X
X_same = df[df['are_same_race'] == 1]
y_same = X_same.is_match
X_same = X_same.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date'])
X_same = pd.get_dummies(X_same) #get dummies for X

# Different race data split into y and x
X_diff = df[df['are_same_race'] == 0]
y_diff = X_diff.is_match
X_diff = X_diff.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date'])
X_diff = pd.get_dummies(X_diff) #get dummies for X

x_train_same, x_test_same, y_train_same, y_test_same = train_test_split(X_same, y_same, test_size = .33,random_state=0)
x_train_diff, x_test_diff, y_train_diff, y_test_diff = train_test_split(X_diff, y_diff, test_size = .33,random_state=0)

clf1 = RandomForestClassifier(n_estimators = 125, random_state=0, min_samples_leaf=2)
clf1.fit(x_train_same, y_train_same) # fit model

clf2 = RandomForestClassifier(n_estimators = 125, random_state=0, min_samples_leaf=2)
clf2.fit(x_train_diff, y_train_diff) # fit model

cross_val_dict = {}
# get the roc auc and pring
# Diff
cross_val_accuracy_roc_auc = (cross_val_score(clf1, x_train_same, y_train_same, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_dict['same'] = cross_val_accuracy_roc_auc

# Diff
cross_val_accuracy_roc_auc = (cross_val_score(clf1, x_train_diff, y_train_diff, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_dict['diff'] = cross_val_accuracy_roc_auc

print(cross_val_dict)


{'same': 82.29589371980677, 'diff': 81.3318054494525}


In [30]:
# Same results
results_same = x_test_same
y_hat = clf1.predict(x_test_same)
results_same['predicted'] = y_hat
results_same['actual'] = y_test_same

# Diff results
results_diff = x_test_diff
y_hat = clf2.predict(x_test_diff)
results_diff['predicted'] = y_hat
results_diff['actual'] = y_test_diff

results = pd.concat([results_same, results_diff])

In [ ]:
results.head()

,is_dater_male,dater_age,dated_age,age_difference,are_same_race,same_race_importance_for_dater,same_religion_importance_for_dater,attractiveness_importance_for_dated,sincerity_importance_for_dated,intelligence_importance_for_dated,...,dater_race_'Latino/Hispanic American',dater_race_caucasian,dater_race_other,dated_race_'Asian/Pacific Islander/Asian-American',dated_race_'Black/African American',dated_race_'Latino/Hispanic American',dated_race_caucasian,dated_race_other,predicted,actual
601,False,34,24,10,True,5.0,7.0,10.0,20.0,35.0,...,False,True,False,False,False,False,True,False,0,0
694,False,30,26,4,True,3.0,5.0,18.0,18.0,18.0,...,False,True,False,False,False,False,True,False,0,0
764,False,29,26,3,True,1.0,2.0,18.0,18.0,18.0,...,False,True,False,False,False,False,True,False,0,0
453,True,28,23,5,True,2.0,2.0,10.0,20.0,20.0,...,False,True,False,False,False,False,True,False,0,0
149,False,27,30,3,True,2.0,3.0,20.0,20.0,25.0,...,False,True,False,False,False,False,True,False,0,0


In [31]:
dater_groups = ["dater_race_'Asian/Pacific Islander/Asian-American'","dater_race_'Black/African American'","dater_race_'Latino/Hispanic American'", 'dater_race_caucasian','dater_race_other']
dated_groups = ["dated_race_'Asian/Pacific Islander/Asian-American'","dated_race_'Black/African American'","dated_race_'Latino/Hispanic American'", 'dated_race_caucasian','dated_race_other']

def safe_divide(n, d):
  try:
    return n/d
  except ZeroDivisionError:
    return np.nan


def compute_metrics(info, group):
  metric_df = info.loc[info[group] == 1]
  pred_pos = sum(metric_df['predicted'])
  act_pos = sum(metric_df['actual'])

  positive_rate = pred_pos/len(metric_df)
  accuracy = len(metric_df[metric_df['predicted'] == metric_df['actual']])/len(metric_df)
  tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
  fp = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 1)])
  fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
  tn = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 0)])
  tpr = safe_divide(tp, (tp + fn))
  fpr = safe_divide(fp, (fp + tn))
  precision = safe_divide(tp, (tp + fp))
  recall = safe_divide(tp, (tp + fn))

  row = [group, accuracy, precision, recall, positive_rate, tpr, fpr]

  return row






In [32]:
column_names = ['group', 'accuracy', 'precision', 'recall', 'positive_rate', 'tpr', 'fpr']
metrics_df2 = pd.DataFrame(columns=column_names)
for group in dater_groups:
  row = compute_metrics(results, group)
  metrics_df2.loc[len(metrics_df2)] = row

for group in dated_groups:
  row = compute_metrics(results, group)
  metrics_df2.loc[len(metrics_df2)] = row

metrics_df2

,group,accuracy,precision,recall,positive_rate,tpr,fpr
0,dater_race_'Asian/Pacific Islander/Asian-Ameri...,0.897436,0.500000,0.125000,0.025641,0.125000,0.014286
1,dater_race_'Black/African American',0.888889,1.000000,0.666667,0.222222,0.666667,0.000000
2,dater_race_'Latino/Hispanic American',0.870968,NaN,0.000000,0.000000,0.000000,0.000000
3,dater_race_caucasian,0.847291,0.750000,0.171429,0.039409,0.171429,0.011905
4,dater_race_other,0.769231,1.000000,0.400000,0.153846,0.400000,0.000000
5,dated_race_'Asian/Pacific Islander/Asian-Ameri...,0.876712,NaN,0.000000,0.000000,0.000000,0.000000
6,dated_race_'Black/African American',0.833333,1.000000,0.500000,0.166667,0.500000,0.000000
7,dated_race_'Latino/Hispanic American',0.826087,0.500000,0.250000,0.086957,0.250000,0.052632
8,dated_race_caucasian,0.857143,0.857143,0.166667,0.032258,0.166667,0.005525
9,dated_race_other,0.812500,0.750000,0.600000,0.250000,0.600000,0.090909


In [33]:
metrics_df.set_index(metrics_df.columns[0], inplace=True)
metrics_df2.set_index(metrics_df2.columns[0], inplace=True)

perc_change_df = percentage_change = safe_divide((metrics_df2 - metrics_df), metrics_df) * 100
perc_change_df

,accuracy,precision,recall,positive_rate,tpr,fpr
group,,,,,,
dater_race_'Asian/Pacific Islander/Asian-American',210.023310,inf,inf,-2.564103,inf,inf
dater_race_'Black/African American',255.555556,100.000000,33.333333,-11.111111,33.333333,NaN
dater_race_'Latino/Hispanic American',291.935484,NaN,NaN,-100.000000,NaN,NaN
dater_race_caucasian,218.889386,87.500000,-5.714286,-49.014778,-5.714286,-67.261905
dater_race_other,438.461538,inf,inf,115.384615,inf,NaN
dated_race_'Asian/Pacific Islander/Asian-American',265.296804,NaN,NaN,-100.000000,NaN,-100.000000
dated_race_'Black/African American',233.333333,50.000000,25.000000,100.000000,25.000000,NaN
dated_race_'Latino/Hispanic American',346.086957,inf,inf,17.391304,inf,inf
dated_race_caucasian,216.981132,157.142857,66.666667,-47.311828,66.666667,-70.718232


## 3 - Post-Processing

In [54]:
threshold_values = [i/100 for i in range(50, 90)]
y_hat_probs = clf.predict_proba(X_test)[:,1]

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- dated_race
- dater_race
Feature names seen at fit time, yet now missing:
- ambition_importance_for_dated
- ambition_importance_for_dater
- ambition_score_of_dater_from_dated
- attractiveness_importance_for_dated
- attractiveness_importance_for_dater
- ...


In [ ]:
def evaluate_group_thresholds(y_true, y_proba, groups, threshold_values):

  unique_groups = np.unique(groups)
  all_combinations = list(product(threshold_values, repeat=len(unique_groups)))
  results = []

  for combination in all_combinations:
    group_thresholds = {group: threshold for group, threshold in zip(unique_groups, combination)}
  
    y_pred = np.zeros(y_true.shape)
    for group, threshold in group_thresholds.items():
      group_mask = (groups == group)
      y_pred[group_mask] = (y_proba[group_mask] > threshold)

    fprs = {}
    for group in unique_groups:
      group_mask = (groups == group)
      group_true = y_true[group_mask]
      group_pred = y_pred[group_mask]
      FP = np.sum((group_pred == 1) & (group_true == 0))
      TN = np.sum((group_pred == 0) & (group_true == 0))
      fprs[group] = FP / (FP + TN)

    overall_accuracy = accuracy_score(y_true, y_pred)
    fpr_diff = fprs[unique_groups[0]] - fprs[unique_groups[1]] 
    row = [combination[0], combination[1], overall_accuracy, fpr_diff]
    results.append(row)
    
    
  columns = ['Threshold 0', 'Threshold 1', 'Model Accuracy', 'FPR Difference']
  results_df = pd.DataFrame(results, columns=columns)
  
  return results_df
results = evaluate_group_thresholds(y_true = y_test, 
                                    y_proba = y_hat_probs, 
                                    groups = X_test['groups'], 
                                    threshold_values = threshold_values)

In [ ]:
fpr_threshold_subset = results[abs(results['FPR Difference']) < 0.01]
max_accuracy_index = fpr_threshold_subset['Model Accuracy'].idxmax()
max_accuracy_row = fpr_threshold_subset.loc[max_accuracy_index]
max_accuracy_row

In [ ]:
def generate_group_level_labels(y_proba, groups, max_accuracy_row):
    y_pred = np.zeros(y_proba.shape)
    for group in groups.unique():
        group_mask = (groups == group)

        threshold = max_accuracy_row['Threshold ' + str(group)]
        
        y_pred[group_mask] = (y_proba[group_mask] > threshold)

    return y_pred

y_hat_thresholds = generate_group_level_labels(y_proba=y_hat_probs,
                                               groups=X_test['are_same_race'],
                                               max_accuracy_row=max_accuracy_row)


    
y_hat_thresholds = generate_group_level_labels(y_proba = y_hat_probs,
                                               groups = X_test['are_same_race'],
                                               max_accuracy_row = max_accuracy_row)
    
    

In [ ]:
metrics_df = compute_metrics(y_hat = y_hat_thresholds,
                             y_test = y_test,
                             X_test = X_test)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))